In [1]:
import pickle as pkl
import numpy as np

import os

from scipy import sparse  as sp

from fastxml import Trainer, Inferencer
from eval_helpers import precision_at_ks

In [2]:
import tensorflow as tf

tf.flags.DEFINE_string('data_dir', 'data/stackexchange/datascience/', 'directory of dataset')
tf.flags.DEFINE_integer('tag_freq_threshold', 5, 'minimum frequency of a tag')

FLAGS = tf.flags.FLAGS
FLAGS._parse_flags()

data_dir = FLAGS.data_dir



In [3]:
X = pkl.load(open(os.path.join(data_dir, "X.pkl"), 'rb'))
Y = pkl.load(open(os.path.join(data_dir, "Y.pkl"), 'rb'))

In [11]:
X.data = np.asarray(X.data, dtype=np.float32)

In [12]:
from sklearn.cross_validation import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, train_size=0.8, random_state=42)

In [15]:
# fastxml

model_path = os.path.join(data_dir, 'fastxml.model')

trainer = Trainer(n_trees=32, n_jobs=-1)

trainer.fit(list(X_train), Y_train)

trainer.save(model_path)

clf = Inferencer(model_path)

pred = clf.predict(X_test)
precision_at_ks(pred, Y_test)

precision at 1: 0.11904362416107381
precision at 3: 0.24400167785234897
precision at 5: 0.305341163310962
precision at 10: 0.4204977628635347


In [16]:
# PFastreXML
from fastxml.weights import propensity

model_path = os.path.join(data_dir, 'pfastrexml.model')

weights = propensity(Y_train)

trainer = Trainer(n_trees=32, n_jobs=-1, leaf_classifiers=True)

trainer.fit(list(X_train), Y_train, weights)

trainer.save(model_path)

clf = Inferencer(model_path)

pred = clf.predict(X_test)
precision_at_ks(pred, Y_test)

precision at 1: 0.11559004474272931
precision at 3: 0.2470357941834452
precision at 5: 0.3138282997762864
precision at 10: 0.427572706935123
